In [1]:
import pandas as pd
from IPython.display import display
import numpy as np
from collections import Counter
import nltk
import psycopg2
from nltk.corpus import stopwords
import re
from itertools import chain 
from IPython.display import Image

In [48]:
conn = psycopg2.connect(host="winkl-production.cpsgstz5cgos.us-west-2.rds.amazonaws.com",port="5432",database="winkl_production", user="winkl", password="Winkl911911")
cur = conn.cursor()

In [17]:
def process(array):
    text = re.sub(r'\[[0-9]*\]',' ',str(array))  #Remove Numbers
    text = re.sub("^\d+\s|\s\d+\s|\s\d+$", " ", text) # Remove nums
    text = re.sub(r'\s+',' ',text)  #Remove extra space
    text = re.sub(r"[^a-zA-Z0-9]+",' ',text)  #Remove special characters
    text = text.lower()  #Lower case all
    text = nltk.sent_tokenize(text)  #Tokenize to sentences 
    keywords = [nltk.word_tokenize(sentence) for sentence in text]
    for i in range(len(keywords)):
        keywords[i] = [word for word in keywords[i] if word not in stopwords.words('english')]
    return keywords

## Analysing profiles of each category | Curating and assigning keywords for each 

### For general information

 ```[[1, "Food"], [2, "Fashion"], [3, "Beauty"], [4, "Lifestyle"], [5, "Travel"], [6, "Photography"], [8, "Make-up"], 
 [7, "Art"], [11, "Gaming"], [12, "Fitness"], [13, "Health"], [14, "Entertainment"], [15, "Gadgets & Tech"], 
 [16, "Finance"], [17, "Luxury"], [18, "Animal/Pet"], [19, "Self Improvement"], [20, "Parenting"], [21, "Books"], 
 [22, "Sports"], [23, "Education"]] ```

### Lifestyle

#### Get top profiles (from insta_categoriesed)

In [10]:
category = 'lifestyle'
cur.execute('''SELECT id, insta_user_id, handle, url, followers, bio FROM "instagram_accounts" WHERE "instagram_accounts"."insta_user_id" IN (SELECT  "insta_category_tags"."id" FROM "insta_category_tags" WHERE (category_tag_id =4 and percentage>15))''')
insta_accounts = pd.DataFrame(cur.fetchall(),columns=["instagram_account_id","insta_user_id","handle","url","followers","bio"])
display(insta_accounts)
insta_accounts.to_csv('E:\Winkl Mains\Task_4_Recategorisation\DATA\\'+ category+'_categorized.csv')

,instagram_account_id,insta_user_id,handle,url,followers,bio
0,60627,29884,paddefoods,https://www.instagram.com/paddefoods/,650,Photos about my #food and #breakfast •#healthy...
1,62629,34764,the.creative.shutter,https://www.instagram.com/the.creative.shutter/,2042,THE CREATIVE SHUTTER by Shivank & Puneet \nWed...
2,100695,53764,paradisemakeover,https://www.instagram.com/paradisemakeover/,1211,MUA\nFreelancer Pro Makeup & Hair Artist..!\nW...
3,193613,63184,trendsetter__collection,https://www.instagram.com/trendsetter__collect...,644,Best jewellery & clothes store❤Customised prod...
4,34762,117404,mallmartonline,https://www.instagram.com/mallmartonline/,1059,Mall Mart Shop🌎Free Worldwide Shipping
5,155717,151984,suhaib_azmi_13,https://www.instagram.com/suhaib_azmi_13/,390,13-feb📷 photography 📷Ex-mintorianBA ( PSYCHOLO...
6,158688,169504,ash_patel143,https://www.instagram.com/ash_patel143/,391,Gujju 💰💸💲Mechanical Engineer⚙️22/06🌍I want to ...
7,193777,205344,mydbali,https://www.instagram.com//mydbali/,184,( Mayadenawa Bali )Just Tap The Link.And Book ...
8,190709,217164,silas_goncalves88,https://www.instagram.com/silas_goncalves88/,1018,Welcome to my Life\nBusiness man \n31 anos\nNa...
9,288912,297044,sev_et_les_petites_betes,https://www.instagram.com/sev_et_les_petites_b...,578,Don’t follow me if U want to unfollow and incr...


#### Get profiles (from curation)

In [11]:
cur.execute('''SELECT id, insta_user_id, handle, url, followers, bio FROM "instagram_accounts" WHERE "instagram_accounts"."insta_user_id" IN (SELECT insta_user_id FROM "curated_insta_user_categories" WHERE "curated_insta_user_categories"."category_tag_id" = 4) LIMIT 20''')
insta_accounts = pd.DataFrame(cur.fetchall(),columns=["instagram_account_id","insta_user_id","handle","url","followers","bio"])
display(insta_accounts)
insta_accounts.to_csv('E:\Winkl Mains\Task_4_Recategorisation\DATA\\'+ category+'_curated.csv')

,instagram_account_id,insta_user_id,handle,url,followers,bio
0,257324,132,vipul_juneja,https://www.instagram.com/vipul_juneja/,294387,"#MensFashion, #MensLifestyle & #Travel Inspira..."
1,49305,134,theleggylass,https://www.instagram.com/theleggylass/,39494,Blogger • Engineer • Army brat\n#Fashion #Beau...
2,117923,135,akanksharedhu,https://www.instagram.com/akanksharedhu/,175584,Personal Style | Travel | Beauty | Luxury\nEma...
3,254470,136,truefashionholic,https://www.instagram.com/truefashionholic/,293066,ʏᴏᴜʀ ᴅᴇsɪ ᴅɪᴠᴀ!\nजीवेषु करुणा चापि मैत्री तेषु...
4,257231,141,ankit.duhan,https://www.instagram.com/ankit.duhan/,34080,23 | JAT | MODEL | | Blogger/influencer | Mr. ...
5,49342,153,marsplayco,https://www.instagram.com/marsplayco/,86604,🧥 Curate. Create. Shop. Earn.\n💃 Community of ...
6,255750,172,i_m_indira,https://www.instagram.com/i_m_indira/,46274,Content Creator\nLifestyle • Fashion • Beauty ...
7,49303,179,amritasahneyy,https://www.instagram.com/amritasahneyy/,61874,"Organic blend of fashion, lifestyle and travel..."
8,188192,187,akshayajay11,https://www.instagram.com/akshayajay11/,3532,•\n@_horizons__photography_ \n🎦YouTube ⬇
9,245251,191,thestylistnut,https://www.instagram.com/thestylistnut/,4761,Fashion & lifestyle blogger \nEngineer🎓\n👻- th...


#### Histogram analysis for curated profile captions

In [25]:
# Run Once | Heavy computation
cur.execute('SELECT insta_user_id, caption FROM insta_user_posts WHERE insta_user_id IN'+str(tuple(insta_accounts["insta_user_id"].tolist())))
captions = pd.DataFrame(cur.fetchall(),columns=["insta_user_id","captions"])
display(captions)
caption_array = ""
for i in captions.index:
    caption_array = caption_array+" "+str(captions["captions"][i])
caption_array = process(caption_array)
print(caption_array)



,insta_user_id,captions
0,132,Always great to feel the energy given from the...
1,132,"Designed for weather, made for style. Loving t..."
2,132,Virtual ride to the space with @kunalaniltanna...
3,132,"#RePost\nThis Christmas, celebrate with the Co..."
4,132,#Repost\nThey say great things just happen out...
...,...,...
5281,223,Aaj khush toh bohot honge aap.. Haiiin❤️..#wax...
5282,223,Bangs❤️..Blazer - @hm Sweater- @koovsfashion ....
5283,223,"Sometimes i wish i could be you, so i could be..."
5284,223,Too much disciplined 😀


[['always', 'great', 'feel', 'energy', 'given', 'ice', 'cold', 'freshness', 'bulgariparfums', 'man', 'glacial', 'essence', 'allows', 'stay', 'creative', 'keep', 'touch', 'nature', 'beautyconcepts', 'india', 'conqueryourgreatness', 'manglacialessence', 'bvlgariparfums', 'designed', 'weather', 'made', 'style', 'loving', 'extremely', 'comfortable', 'functional', 'cross', 'body', 'bag', 'aldo', 'shoes', 'aldocrew', 'aldoindia', 'aldoxvipuljuneja', 'virtual', 'ride', 'space', 'kunalaniltanna', 'collection', 'reels', 'reel', 'reelsinstagram', 'reel', 'repost', 'christmas', 'celebrate', 'cool', 'blue', 'merrychristmas', 'happynewyear', 'imperialblue', 'thenewblue', 'thecoolblue', 'grainnahitohwhiskynahi', 'ektohbantahai', 'whisky', 'whiskylover', 'bar', 'whiskygram', 'drinks', 'instawhisky', 'callsforanimperialblue', 'seagrams', 'goodspiritsdelivered', 'imperialblueindia', 'repost', 'say', 'great', 'things', 'happen', 'blue', 'let', 'bring', 'new', 'blue', 'happy', 'new', 'year', 'imperialblu

In [40]:
# Histogram Code

keywords = []
caption_freq = []
counts = Counter(caption_array[0])

labels, values = zip(*counts.items())
## sort your values in descending order
indSort = np.argsort(values)[::-1]
## rearrange your data
keywords = np.array(labels)[indSort]  # Label
caption_freq = np.array(values)[indSort]  # Values
    
row = {'word':keywords, 'count':caption_freq}
hist = pd.DataFrame(row)
display(hist[-50:-5])

,word,count
22930,pahiyon,1
22931,canwerewindthetimenow,1
22932,lifestyleproducts,1
22933,koreanproducts,1
22934,haulvideo,1
22935,mumusohaul,1
22936,weavingdreams,1
22937,foliate,1
22938,intertwining,1
22939,zari,1


### Manual curated keywords

In [37]:
lifestyle_keyword = ["blogger","style","ootd","outfitoftheday"]

## Education

In [41]:
category = 'fitness'
cur.execute('''SELECT id, insta_user_id, handle, url, followers, bio FROM "instagram_accounts" WHERE "instagram_accounts"."insta_user_id" IN (SELECT insta_user_id FROM "curated_insta_user_categories" WHERE "curated_insta_user_categories"."category_tag_id" = 12) LIMIT 20''')
insta_accounts = pd.DataFrame(cur.fetchall(),columns=["instagram_account_id","insta_user_id","handle","url","followers","bio"])
display(insta_accounts)
insta_accounts.to_csv('E:\Winkl Mains\Task_4_Recategorisation\DATA\\'+ category+'_curated.csv')

,instagram_account_id,insta_user_id,handle,url,followers,bio
0,257324,132,vipul_juneja,https://www.instagram.com/vipul_juneja/,290860,"#MensFashion, #MensLifestyle & #Travel Inspira..."
1,255342,478,ramneeksodhi0,https://www.instagram.com/ramneeksodhi0/,10588,Myglamm \nhttps://www.myglamm.com/myglammxo-su...
2,49441,527,deep.hora,https://www.instagram.com/deep.hora/,18226,Live it up. \n📍 Punjab
3,197057,682,abhimanika,https://www.instagram.com/abhimanika/,15378,Fashion | Fitness \nMrs.Universe2017\nMrs.Asia...
4,255182,715,psbypiyushsharma,https://www.instagram.com/psbypiyushsharma/,13439,"It's not about the destination, it's about the..."
5,195774,829,herhappyface,https://www.instagram.com/herhappyface/,27359,Managing a Full Time Job + YouTube! 💪🏻\nSpread...
6,49700,1290,hypertroph,https://www.instagram.com/hypertroph/,10147,Fitness & Nutrition Coach 💪\nYoutuber🎥\nIIT Gr...
7,49747,1417,naturallysweebs,https://www.instagram.com/naturallysweebs/,9395,⊹ fitness x health x foodie\n⊹ doxie mom @navy...
8,254525,1509,little__curves,https://www.instagram.com/little__curves/,8804,Certified Dietitian/Paediatric Nutritionist \n...
9,258063,1780,reetvlogs2019,https://www.instagram.com/reetvlogs2019/,10888,"Influencer, Lifestyle vlogger\n🎥Youtube-REET V..."


In [50]:
# cur.execute("""SELECT "users"."id" FROM "users" WHERE "users"."city" = 'Hyderabad'""")
cur.execute("""SELECT handle,url,followers,avg_engagement FROM instagram_accounts WHERE user_id IN (SELECT id from users WHERE city = 'Hyderabad')""")
hyd = pd.DataFrame(cur.fetchall(),columns=["handle","url","followers","avg_engagement"])
display(hyd)
hyd.to_csv('E:\Winkl Mains\Task_4_Recategorisation\DATA\hyd_data.csv')

,handle,url,followers,avg_engagement
0,classiestindiangirls,https://www.instagram.com/classiestindiangirls/,308.0,2.8085
1,abigsmilegirl,https://www.instagram.com/abigsmilegirl/,196.0,0.0000
2,vaishakhi_here,https://www.instagram.com/vaishakhi_here/,1137.0,4.5700
3,omyadav7687,https://www.instagram.com/omyadav7687/,163.0,43.4000
4,teavins,https://www.instagram.com/teavins/,8225.0,3.2900
...,...,...,...,...
315,gentle_man_wardrobe,https://www.instagram.com/gentle_man_wardrobe/,19844.0,10.9300
316,places_food_explorer,https://www.instagram.com/places_food_explorer/,207.0,10.3300
317,reality_bite,https://www.instagram.com/reality_bite/,68775.0,1.6100
318,makeuporino,https://www.instagram.com/makeuporino/,2526.0,1.1800
